In [1]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

from functools import partial
from torch.utils.data import DataLoader

from problems import *
# from src_2d.data import OfflineDataset, OnlineDataset, custom_collate_fn
# from src_2d.model import DPT
from src_1d.data import OfflineDataset, OnlineDataset, custom_collate_fn
from src_1d.model import DPT

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
n = 2
d = 5

# continuous problems are not supported yet
if n == 1:
    problem_class = ContinuousQuadratic
    problems = [problem_class(d=d, n=n, seed=seed) for seed in range(10)]
else:
    # problem_class = DiscreteQuadratic
    # problems = [problem_class(d=d, n=n, seed=seed) for seed in range(10)]

    problem_class = DiscretePolynomial
    problems = [problem_class(d=d, n=n, m=3, seed=seed) for seed in range(10)]

In [ ]:
seq_len = 3
batch_size = 2

dataset = OfflineDataset(problems, seq_len=seq_len)
collate_fn = partial(custom_collate_fn, problem_class=problem_class)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
batch = next(iter(dataloader))
batch

In [ ]:
print('query_state', batch["query_state"].shape, batch["query_state"].dtype)
print('states', batch["states"].shape, batch["states"].dtype)
print('actions', batch["actions"].shape, batch["actions"].dtype)
print('next_states', batch["next_states"].shape, batch["next_states"].dtype)
print('rewards', batch["rewards"].shape, batch["rewards"].dtype)
print('target_action', batch["target_action"].shape, batch["target_action"].dtype)
print('problem', batch["problem"], type(batch["problem"]))

In [ ]:
model = DPT(
    state_dim=d+1, action_dim=d+1, 
    seq_len=seq_len, hidden_dim=16,
    embedding_dropout=0.0, attention_dropout=0.0, residual_dropout=0.0
)
outputs = model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=batch["rewards"]
)
outputs

In [ ]:
print('outputs', outputs.shape, outputs.dtype, outputs.device)

In [ ]:
outputs.sum(-1)